# Тестирование сжатия в классе DataSamples

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import string
import ipytest
import pytest
import sys

sys.path.append("/mnt/d/repo/packages/")
from vtb_mlkit.scorekit import DataSamples
import vtb_mlkit

ipytest.autoconfig()

pd.set_option("display.float_format", lambda x: "%.5f" % x)


(CVXPY) Jul 19 09:17:40 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jul 19 09:17:40 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')


In [2]:
# проверка, что фреймворк нужной версии
assert vtb_mlkit.__version__ == "1.0.0"

Возможные кейсы:
* столбец с числами float64, которые не должны быть сжаты;
* столбец с числами float32, но представнленные float64, которые должны быть сжаты до float32;
* столбец с числами int64, которые не должны быть сжаты;
* столбец с числами int32, но представнленные int64, которые должны быть сжаты до int32;
* столбец с числами int16, но представнленные int64, которые должны быть сжаты до int16;
* столбец с числами int8,  но представнленные int64, которые должны быть сжаты до int8;
Дополнительно создаем столбцы, содержащие спец. значения
* Спец. значения 9999999, 999999999, -999999999,-9999999. При сжатии не должны потерять в точности.
    * в столбце с пропусками определяются как float64;
    * в столбце без пропусков определяются как int64;

In [15]:
int(2.9)

2

In [12]:
999.6 == approx(998.7, abs=0.99999)

True

## Собираем выборку для тестов

In [16]:
# дополнительный набор
additional_values = [
    [-999.5, np.nan, 0.0, 998.5, 999.0],
    [
        np.nan,
        -9_999_999.0,
        9_999_999.0,
        -7_777_777.0,
        7_777_777.0,
    ],
    [
        np.nan,
        -999_999_999.0,
        999_999_999.0,
        -7_777_777.0,
        7_777_777.0,
    ],
    [0, 2, 3, 4, 5],
    [0, 22_222, 3, 4, 5],
    [
        0,
        -999_999_999,
        999_999_999,
        -7_777_777,
        7_777_777,
    ],
    ["a", "A", "ZTR", "wer", "dfsa dfs"],
]


In [17]:
# словарь с пограничными значениями для генерации случайных чисел
limits_dict = {
    "f16_limit": 10**3 - 1,
    "f32_limit": 10**7 - 1,
    "f64_limit": 440282346638528859811704183484516925440.0,
    "i8_limit": 120,
    "i16_limit": 32700,
    "i32_limit": 2147483600,
    "i64_limit": 9223372036854775000,
}


In [18]:
# фиксируем генерацию случайных чисел для воспроизводимости
np.random.seed(42)
# создаем набор случайных значений, дополнив выбранными
data_dict = {
    "to_float16": np.append(
        np.random.uniform(-limits_dict["f16_limit"], limits_dict["f16_limit"], [1, 20])[
            0
        ],
        additional_values[0],
    ),  # числа, которые будут сжаты в float16
    "to_float32": np.append(
        np.random.uniform(-limits_dict["f32_limit"], limits_dict["f32_limit"], [1, 20])[
            0
        ],
        additional_values[1],
    ),  # числа, которые будут сжаты в float32
    "to_float64": np.append(
        np.random.uniform(-limits_dict["f64_limit"], limits_dict["f64_limit"], [1, 20])[
            0
        ],
        additional_values[2],
    ),  # числа, которые не должны быть сжаты
    "to_int8": np.append(
        np.random.randint(-limits_dict["i8_limit"], limits_dict["i8_limit"], 20),
        additional_values[3],
    ),  # числа, которые будут сжаты в int8
    "to_int16": np.append(
        np.random.randint(-limits_dict["i16_limit"], limits_dict["i16_limit"], 20),
        additional_values[4],
    ),  # числа, которые будут сжаты в int16
    "to_int32": np.append(
        np.random.randint(-limits_dict["i32_limit"], limits_dict["i32_limit"], 20),
        additional_values[5],
    ),  # числа, которые будут сжаты в int32
    "to_int64": np.append(
        np.random.randint(-limits_dict["i64_limit"], limits_dict["i64_limit"], 20),
        additional_values[5],
    ),  # числа, которые не должны быть сжаты
    "to_category": np.append(
        np.random.choice(list(string.ascii_uppercase), 20), additional_values[6]
    ),
}


* Баг, что метод DataSamples перезаписывает подаваемый датафрейм.

In [22]:
original_df = pd.DataFrame(data_dict)
original_df["target_int8"] = original_df["to_int8"].copy()
original_df


,to_float16,to_float32,to_float64,to_int8,to_int16,to_int32,to_int64,to_category,target_int8
0,-250.66884,2237057.67074,-332819785804362643784273142287706357760.00000,87,29892,1503404280,8701552265281353973,S,87
1,900.52718,-7210122.06595,-4247042668517015323795297186101592064.00000,116,-30673,529561463,6436323927311700520,V,116
2,463.52390,-4157106.61358,-410001029090454956392547676585782673408.00000,-39,-30005,-217107653,4090187672963768321,B,-39
3,197.11965,-2672762.86685,360433094308544611908576024096692764672.00000,-10,26139,-726287407,-4870218625151263194,J,-10
4,-687.27476,-878600.22780,-212409831514667308241523277115145846784.00000,-68,15490,-1737700272,-4499745231403915634,M,-68
5,-687.32295,5703518.65751,143111385472850871466416075971613425664.00000,-97,-27442,-1874502561,-8477503916587494224,Y,-97
6,-882.94894,-6006523.75618,-165800578530795110455006623647183929344.00000,33,-10698,-554831322,3886044415066855222,U,33
7,731.61994,284688.73980,17671190913116481358137087070081908736.00000,96,6804,-811824698,-7177797341225557679,F,96
8,202.02779,1848291.19241,41131422802800695820968167417569083392.00000,67,459,725167725,-1119043923099072268,L,67
9,415.72901,-9070990.83850,-277506039707795602440480052271374139392.00000,3,-18714,-750831865,-5502309463496081050,L,3


In [23]:
compressed_df = DataSamples(
    samples={
        "train": original_df.copy()
    },  # здесь copy из-за перезаписи подаваемого датафрейма
    compress=True,
    cat_columns=["to_category"],
    target="target_int8",
).samples["train"]


[INFO] [2023-07-19 10:07:49] ---------------------------------------------------------------- Creating DataSamples ----------------------------------------------------------------
[INFO] [2023-07-19 10:07:49] Selected 8 features: ['to_float16', 'to_float32', 'to_float64', 'to_int8', 'to_int16', 'to_int32', 'to_int64', 'to_category']
[INFO] [2023-07-19 10:07:49] DataSamples stats:
               train
amount            25
target          -191
target_rate -7.64000
period            NA


In [40]:
not all(
    col in original_df.columns
    for col in compressed_df.dtypes.astype("str").reset_index()["index"]
)


True

## Тесты

In [42]:
%%ipytest -qq

# выборка после сжатия
@pytest.fixture(scope='session')
def get_compressed_df():
    ds = DataSamples(
        samples={"train": original_df.copy()},
        compress=True,
        cat_columns=['to_category'],
        target='target_int8'
    )
    return ds.samples['train']

@pytest.fixture(scope='session')
def get_original_df():
    return original_df.copy()

# Smoke test
def test_compress_data(get_original_df):
    DataSamples(
        samples={"train": get_original_df.copy()},
        compress=True,
        cat_columns=['to_category'],
        target='target_int8'
    )
    assert True


def test_columns_dtypes(get_compressed_df):
    """
    Функция для тестирования типов данных в столбцах
    после сжатия. В названии столбца зашит предполагаемый
    тип данных после сжатия, например: to_float16 - это float16
    """
    # задаем датафрейм с названиями столбцов и типом данных
    columns_dtypes_df = get_compressed_df.copy().dtypes.astype('str').reset_index()
    # отбираем данные по условию, что
    # название столбца соответствует ожидаемому типу
    condition_dtypes = columns_dtypes_df.apply(lambda row: row['index'].split('_')[1] == row[0], 1)
    # не соответствующие типу названию
    # столбцы записываем в отдельную переменную
    not_match_columns = np.array2string(columns_dtypes_df[condition_dtypes == False]['index'].values, separator=',')
    msg = f'Столбцы, которые не соответствуют типу: {not_match_columns}'
    assert condition_dtypes.all(), msg

def test_int_and_cat_values_match(get_compressed_df, get_original_df, abs_tol=1e-4):
    """
    Функция для тестирования того, что после
    сжатия данные не теряют в точности
    Parameters:
    get_compressed_df : датафрейм со сжатыми данными
    get_original_df : исходный датафрейм
    abs_tol : абсолютная точность (по умолчания 0.0001)
    """
    df_compressed = get_compressed_df.copy()
    df_original = get_original_df.copy()

    selected_columns = ['to_int8', 'to_int16', 'to_int32', 'to_int64', 'to_category']

    for column in selected_columns:
        
        msg = f'Значения в столбце {column} не соответствуют'
        # если тип int или object, то сравниваем "в лоб"        
        if pd.api.types.is_integer_dtype(df_original[column]) \
        or df_original[column].dtypes in ['object', 'category']:
            if (df_original[column] == df_compressed[column]).all():
                continue
            else:
                assert False, msg  
        else:
            assert False, f'неизвестный тип в столбце{column}'
    
    assert True

def test_float_values(get_compressed_df, get_original_df):
    """
    Функция для тестирования того, что после
    сжатия данные не теряют в точности
    Parameters:
    get_compressed_df : датафрейм со сжатыми данными
    get_original_df : исходный датафрейм
    abs_tol : абсолютная точность (по умолчания 0.0001)
    """

    def compare_float_values_by_integer(original:float, compressed:float) -> bool:
        """
        функция сравнивает числа с плавающей запятой
        по целой части, если не nan. Если оба значения
        nan, то возвращается True
        """
        # если оба nan
        if pd.isna(original) and pd.isna(compressed):
            return True
        # если только одно из чисел nan
        elif pd.isna(original) or pd.isna(compressed):
            return False
        # в остальных случаях предполагается, что будут только числа
        else:
            int(original) == int(compressed)


    df_compressed = get_compressed_df.copy()
    df_original = get_original_df.copy()

    for column in ['to_float16', 'to_float32', 'to_float64']:
        
        msg = f'Значения в столбце {column} не соответствуют'

        # если тип float, то сравниваем по целому значению
        if pd.api.types.is_float_dtype(df_original[column]) \
            & pd.api.types.is_float_dtype(df_compressed[column]):
            
            # объединим столбцы в один DataFrame и произведем сравнение
            assert pd.merge(df_original[column], df_compressed[column],
                            left_index=True, right_index=True,
                            suffixes=("_original", "_compressed")) \
                     .apply(lambda row: compare_float_values_by_integer(row[column+'_original'],
                                                                        row[column+'_compressed']),
                            axis=1).all(), msg
        
        else:
            assert False, f'неизвестный тип в столбце {column}'
    
    assert True

....                                                                                         [100%]
========================================= warnings summary =========================================
t_56882c00edb04e649736e33913e9e95a.py::test_compress_data
t_56882c00edb04e649736e33913e9e95a.py::test_columns_dtypes
t_56882c00edb04e649736e33913e9e95a.py::test_columns_dtypes
t_56882c00edb04e649736e33913e9e95a.py::test_float_values
t_56882c00edb04e649736e33913e9e95a.py::test_float_values
t_56882c00edb04e649736e33913e9e95a.py::test_float_values
t_56882c00edb04e649736e33913e9e95a.py::test_float_values
  /home/amega/miniconda3/envs/vtb_scorekit_1_2_2/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
  See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
    return np.find_common_type(types, [])

-- Docs: https://

## Анализ расхождений

In [43]:
def calc_abs_diff(original_sample: pd.Series, compressed_sample: pd.Series) -> None:
    """
    Функция рассчитаывает разницу между
    сравниваемыми выборками по модулю
    и выводит результат в виде датафрейма
    """
    print("----" + original_sample.name + "----")
    res_df = pd.merge(
        original_sample,
        compressed_sample,
        left_index=True,
        right_index=True,
        suffixes=("_original", "_compressed"),
    )
    res_df["diff"] = (res_df[res_df.columns[0]] - res_df[res_df.columns[1]]).abs()
    display(res_df)
    print("--end---")


In [44]:
[
    calc_abs_diff(original_df[col], compressed_df[col])
    for col in [x for x in original_df.columns if "float" in x]
]


----to_float16----


,to_float16_original,to_float16_compressed,diff
0,-250.66884,-250.62500,0.04384
1,900.52718,900.50000,0.02718
2,463.52390,463.50000,0.02390
3,197.11965,197.12500,0.00535
4,-687.27476,-687.50000,0.22524
5,-687.32295,-687.50000,0.17705
6,-882.94894,-883.00000,0.05106
7,731.61994,731.50000,0.11994
8,202.02779,202.00000,0.02779
9,415.72901,415.75000,0.02099


--end---
----to_float32----


,to_float32_original,to_float32_compressed,diff
0,2237057.67074,2237057.75000,0.07926
1,-7210122.06595,-7210122.00000,0.06595
2,-4157106.61358,-4157106.50000,0.11358
3,-2672762.86685,-2672762.75000,0.11685
4,-878600.22780,-878600.25000,0.02220
5,5703518.65751,5703518.50000,0.15751
6,-6006523.75618,-6006524.00000,0.24382
7,284688.73980,284688.75000,0.01020
8,1848291.19241,1848291.25000,0.05759
9,-9070990.83850,-9070991.00000,0.16150


--end---
----to_float64----


,to_float64_original,to_float64_compressed,diff
0,-332819785804362643784273142287706357760.00000,-332819785804362643784273142287706357760.00000,0.00000
1,-4247042668517015323795297186101592064.00000,-4247042668517015323795297186101592064.00000,0.00000
2,-410001029090454956392547676585782673408.00000,-410001029090454956392547676585782673408.00000,0.00000
3,360433094308544611908576024096692764672.00000,360433094308544611908576024096692764672.00000,0.00000
4,-212409831514667308241523277115145846784.00000,-212409831514667308241523277115145846784.00000,0.00000
5,143111385472850871466416075971613425664.00000,143111385472850871466416075971613425664.00000,0.00000
6,-165800578530795110455006623647183929344.00000,-165800578530795110455006623647183929344.00000,0.00000
7,17671190913116481358137087070081908736.00000,17671190913116481358137087070081908736.00000,0.00000
8,41131422802800695820968167417569083392.00000,41131422802800695820968167417569083392.00000,0.00000
9,-277506039707795602440480052271374139392.00000,-277506039707795602440480052271374139392.00000,0.00000


--end---


[None, None, None]

## Выводы

Расхождений в целой части чисел с плавающей запятой не выявлено